# Texture prediction

## Evaluation

### Good examples:

-projects/if-net_texture/experiments/shapenet_channel/evaluation_278/bf9ea87a9765eed7d53b73fe621a84b4


## Data preparation
The input dataset currently goes through multiple pre processing steps

In [14]:
# step -1: convert the obj to a compressed numpy array
import numpy as np

filePath = "dataset/SHARP2020/challenge1-track1/test/170410-001-a-r9iu-494a-low-res-result/170410-001-a-r9iu-494a-low-res-result_normalized.npz"

b = np.load(filePath, allow_pickle=True)

print(list(b.keys()))

for key in b.keys():
    print(key)
    print(b[key].shape)

['vertices', 'faces', 'normals', 'texcoords', 'texcoords_indices', 'texture', 'vertex_colors', 'face_vertices']
vertices
(9968, 3)
faces
(20000, 3)
normals
()
texcoords
(13857, 2)
texcoords_indices
(20000, 3)
texture
(2048, 2048, 3)
vertex_colors
()
face_vertices
(106,)


In [1]:
import numpy as np
path = 'projects/if-net_texture/dataset/SHARP2020/chairs/test/170410-001-a-r9iu-494a-low-res-result/170410-001-a-r9iu-494a-low-res-result_normalized_color_samples100000_bbox-7,7,-1,20,-7,7.npz'
split = np.load(path[24:])
print(split.files)

for key in split.files:
    print(key)
    print(split[key])
    print(np.unique(split[key]))

['points', 'grid_coords', 'colors']
points
[[ 0.12064141  0.88453791 -0.24881006]
 [-0.1544254   0.68709336 -0.25824036]
 [-0.18368903  0.25557288 -0.13980684]
 ...
 [-0.38154646  1.22621116 -0.16947448]
 [ 0.00948102  1.06471689 -0.31133876]
 [-0.4635408   0.98523597 -0.12528596]]
[-0.50958245 -0.50924814 -0.50912306 ...  1.78317587  1.78319532
  1.78338817]
grid_coords
[[-0.03554429 -0.8205202   0.01723449]
 [-0.03689148 -0.83932444 -0.02206077]
 [-0.01997241 -0.88042163 -0.02624129]
 ...
 [-0.02421064 -0.78797989 -0.05450664]
 [-0.04447697 -0.8033603   0.00135443]
 [-0.01789799 -0.81092991 -0.06622011]]
[-0.9038141  -0.903814   -0.90381344 ...  0.04831714  0.04831819
  0.04835757]
colors
[[44 50 59]
 [44 58 69]
 [43 66 83]
 ...
 [41 62 81]
 [29 52 94]
 [49 54 68]]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  5

In [1]:
import models.local_model as model
import models.dataloader as dataloader
import argparse
from models.generation import Generator
import config.config_loader as cfg_loader
import trimesh
import torch
from data_processing import utils

In [2]:
cfg = cfg_loader.load("config/SHARP2020/track1.yaml")
net = model.get_models()[cfg['model']]()
gen = Generator(net, cfg)

Conv3d(4, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), padding_mode=reflect)
Loaded checkpoint from: /home/jvermandere/projects/if-net_texture/models/../experiments/SHARP2020_c1_t1/checkpoints/checkpoint_epoch_99.tar


In [8]:
import numpy as np
path = "dataset/SHARP2020/challenge1-track1/test/170410-001-a-r9iu-494a-low-res-result/170410-001-a-r9iu-494a-low-res-result_normalized-partial-00_voxelized_colored_point_cloud_res128_points100000_bbox-0.8,0.8,-0.15,2.1,-0.8,0.8.npz"
inputResolution = cfg['input_resolution']

R = np.load(path)['R']
G = np.load(path)['G']
B = np.load(path)['B']
S = np.load(path)['S']

R = np.reshape(R, (inputResolution,)*3)
G = np.reshape(G, (inputResolution,)*3)
B = np.reshape(B, (inputResolution,)*3)
S = np.reshape(S, (inputResolution,)*3)
input = np.array([R,G,B,S])

inputs = torch.from_numpy(np.array(input, dtype=np.float32))

In [10]:
meshPath = "dataset/SHARP2020/challenge1-track1/test/170410-001-a-r9iu-494a-low-res-result/170410-001-a-r9iu-494a-low-res-result_normalized.obj"
mesh = trimesh.load(meshPath)
        
# create new uncolored mesh for color prediction
pred_mesh = trimesh.Trimesh(mesh.vertices, mesh.faces)

# colors will be attached per vertex
# subdivide in order to have high enough number of vertices for good texture representation
pred_mesh = pred_mesh.subdivide().subdivide()

pred_verts_gird_coords = utils.to_grid_sample_coords( pred_mesh.vertices, cfg['data_bounding_box'])
pred_verts_gird_coords = torch.tensor(pred_verts_gird_coords).unsqueeze(0)

colors_pred_surface = gen.generate_colors(inputs, pred_verts_gird_coords)

# attach predicted colors to the mesh
pred_mesh.visual.vertex_colors = colors_pred_surface

RuntimeError: grid_sampler(): expected grid and input to have same batch size, but got input with sizes [4, 128, 128, 128] and grid with sizes [1, 1, 7, 159968, 3]

In [6]:
import numpy as np

data = np.load("dataset/SHARP2020/challenge1-track1/test/170410-001-a-r9iu-494a-low-res-result/170410-001-a-r9iu-494a-low-res-result_normalized-partial-01.npz")
print(data['vertices'])

[[ 0.13770886  0.099706   -0.12823488]
 [ 0.1605673   0.11850667 -0.20837104]
 [ 0.1467137   0.24583559 -0.23430079]
 ...
 [-0.27044937  0.9719703  -0.14337961]
 [-0.12032622  1.042221    0.00904251]
 [ 0.03044558  1.0324756  -0.30844074]]
